<a href="https://colab.research.google.com/github/SanteriMan/freecodecamp_ML_projects/blob/master/Copy_of_fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing import sequence


print(tf.__version__)

In [299]:
# get data files
TRAIN_DATA_URL = "https://raw.githubusercontent.com/beaucarnes/fcc_python_curriculum/master/sms/train-data.tsv"
TEST_DATA_URL = "https://raw.githubusercontent.com/beaucarnes/fcc_python_curriculum/master/sms/valid-data.tsv"

train_file_path = tf.keras.utils.get_file("train-data.tsv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("valid-data.tsv", TEST_DATA_URL)

train_data = pd.read_csv(train_file_path, sep="\t", names=["label","message"])
test_data = pd.read_csv(test_file_path, sep="\t", names=["label","message"])

train_data.loc[:,'label'] = train_data.label.map({'ham':0, 'spam':1})
test_data.loc[:,'label'] = test_data.label.map({'ham':0, 'spam':1})
train_labels = train_data.pop('label')
test_labels = test_data.pop('label')

train_labels = train_labels.to_numpy()
test_labels = test_labels.to_numpy()

vocab = {}  
word_encoding = 1
def one_hot_encoding(text):
  global word_encoding

  words = text.lower().split(" ") 
  encoding = []  

  for word in words:
    if word in vocab:
      code = vocab[word]  
      encoding.append(code) 
    else:
      vocab[word] = word_encoding
      encoding.append(word_encoding)
      word_encoding += 1
  
  return encoding


for i in range(len(train_data)):
  train_data['message'][i] = list(one_hot_encoding(train_data['message'][i]))
for i in range(len(test_data)):
  test_data['message'][i] = list(one_hot_encoding(test_data['message'][i]))

train_data = train_data['message'].to_numpy()
test_data = test_data['message'].to_numpy()

train_data = sequence.pad_sequences(train_data, 50)
test_data = sequence.pad_sequences(test_data, 50)


In [300]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(88400, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [301]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
105/105 [==============================] - 3s 27ms/step - loss: 0.3808 - acc: 0.8575 - val_loss: 0.1358 - val_acc: 0.9593
Epoch 2/10
105/105 [==============================] - 1s 12ms/step - loss: 0.1109 - acc: 0.9769 - val_loss: 0.0626 - val_acc: 0.9833
Epoch 3/10
105/105 [==============================] - 1s 12ms/step - loss: 0.0475 - acc: 0.9900 - val_loss: 0.0490 - val_acc: 0.9856
Epoch 4/10
105/105 [==============================] - 1s 12ms/step - loss: 0.0300 - acc: 0.9902 - val_loss: 0.0462 - val_acc: 0.9880
Epoch 5/10
105/105 [==============================] - 1s 12ms/step - loss: 0.0185 - acc: 0.9942 - val_loss: 0.0575 - val_acc: 0.9833
Epoch 6/10
105/105 [==============================] - 1s 12ms/step - loss: 0.0155 - acc: 0.9953 - val_loss: 0.0589 - val_acc: 0.9844
Epoch 7/10
105/105 [==============================] - 1s 12ms/step - loss: 0.0110 - acc: 0.9949 - val_loss: 0.0551 - val_acc: 0.9844
Epoch 8/10
105/105 [==============================] - 1s 12ms/step - 

In [302]:
def encode_text(text):

  text = one_hot_encoding(text)

  text = [np.asarray(text)]

  text = sequence.pad_sequences(text, 50)
  return text

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)


[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0   144  1210   243   148 13486
    165  3071]]


In [303]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  encoded = encode_text(pred_text)
  prediction = []
  predictionvalue = model.predict(encoded)[0]
  prediction.append(predictionvalue[0])
  if predictionvalue < 0.5:
    prediction.append('ham')
  else:
    prediction.append('spam')

  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

[7.85976e-05, 'ham']


In [298]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!
